In [1]:
import findspark
findspark.init()

# import pyspark
import sys
import re
import random

from similaritycos import cosine_similarity
#  jaccard_similarity, 
from pyspark import SparkConf, SparkContext

def parseVector(line):
    # line.split("::")
    return line.split(r'::')

def parseRatings(line):
    # line.split("::")moviePairSimilarities.saveAsTextFile("movie-sims")
    fields= parseVector(line)
    return int(fields[0]),(int(fields[1]),float(fields[2]))

def parseMovies(line,movieDict):
    fields= parseVector(line)
    movieDict[int(fields[0])] = fields[1]
    return fields[0],(fields[1],fields[2])

def getSampleInteractions(user, film, n):
    if len(film) > n:
        return user, random.sample(film,n)
    else:
        return user, film
    return

def uniqueFilter((userid, ratings)):
    (item1, value1) = ratings[0]
    (item2, value2) = ratings[1]
    return item1 < item2

def itemitem((item,values)):
    (itemA, valueA) = values[0]
    (itemB, valueB) = values[1]
    return ((itemA, itemB), (valueA, valueB))

if __name__=="__main__":
    if len(sys.argv)< 3:
        print >> sys.stderr, "Usage: MovieLens ratings movies"
        exit(-1)
  #  print "hello"

    ratings_file = sys.argv[1]
    movies_file = sys.argv[2]

    print "ratings {0}\nmovies {1}".format(ratings_file,movies_file)
    sc = SparkContext(appName = "MovieLens")

    movies_data = sc.textFile(movies_file)
    movieDict={}
    movies= movies_data.map(lambda line: parseMovies(line,movieDict))

    ratings_data = sc.textFile(ratings_file)

    ratings = ratings_data.map(parseRatings)


    user_ratings_data= ratings.join(ratings)
    uniqueJoin = user_ratings_data.filter(uniqueFilter)

    moviePairs = uniqueJoin.map(itemitem)

    moviePairRatings = moviePairs.groupByKey()

    movie_movie_Similarities = moviePairRatings.mapValues(cosine_similarity).cache()
    movie_movie_Similarities.sortByKey()
    movie_movie_Similarities.saveAsTextFile("movie-sims1")

ratings -f
movies /run/user/29343/jupyter/kernel-3ff28751-4599-4db2-b793-719ef874ab33.json


Py4JJavaError: An error occurred while calling o51.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: hdfs://studoop.eecs.qmul.ac.uk:8020/user/ao309/-f
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:202)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
	at scala.Option.getOrElse(Option.scala:120)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:237)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
	at scala.

In [ ]:
if (len(sys.argv)> 1):
    scoreThreshold = 0.97
    coOccurenceTHreshold = 50

In [ ]:
movieID = int(sys.argv[1])
filteredResults = movie_movie_Similarities.filter(lambda((pair,sim)): \ (pair[0] == movieID or pair[1] == movieID) \ and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold)

In [ ]:
results = filteredResults.map(lambda((pair,sim)): (sim,
pair)).sortByKey(ascending = False).take(10)